# Projekt MAGN

### Autorzy:

- Szymon Jurecki
- Dominik Breksa

In [1]:
%pip install .
%pip install pandas
%pip install numpy
%pip install kaggle

Processing c:\users\domin\pycharmprojects\ggsn---magn
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for magn: filename=magn-1.0.0.0-py3-none-any.whl size=16641 sha256=a4998373157a2a35a8ef5cebd55097c3a998a8a66d29b83a3bba85bf677aa9d0
  Stored in directory: c:\users\domin\appdata\local\pip\cache\wheels\02\69\7a\7a321fc99cdf850f17c5dbe96e53ef1e4b07adda9d5bd3d1f7
Successfully built magn
  Attempting uninstall: magn
    Found existing installation: magn 1.0.0.0
    Not uninstalling magn at c:\users\domin\pycharmprojects\ggsn---magn\src, outside environment C:\Users\domin\PycharmProjects\GGSN--

DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Pobranie oraz Przygotowanie Danych

In [2]:
from pathlib import Path
from typing import Final

resources_dir: Final[Path] = Path('resources/data/')

zip_data_file_movie: Final[Path] = resources_dir.joinpath('movie-data-analytics-dataset.zip')
database_path_movie: Final[Path]  = resources_dir.joinpath('movie.sqlite')

zip_data_file_pitchfork: Final[Path] = resources_dir.joinpath('pitchfork-data.zip')
database_path_pitchfork: Final[Path]  = resources_dir.joinpath('database.sqlite')

Pobranie danych z platformy Kaggle

[Link do danych z platformy Kaggle](https://www.kaggle.com/datasets/shahjhanalam/movie-data-analytics-dataset)

[Link do danych z platformy Kaggle](https://www.kaggle.com/datasets/nolanbconaway/pitchfork-data)

In [3]:
from os.path import exists

if not exists(zip_data_file_movie):
    !kaggle datasets download -d shahjhanalam/movie-data-analytics-dataset -p resources/data
    
if not exists(zip_data_file_pitchfork):
    !kaggle datasets download -d nolanbconaway/pitchfork-data -p resources/data

Rozpakowanie danych do odpowiedniego katalogu

In [4]:
from zipfile import ZipFile

if not exists(database_path_movie):
    with ZipFile(zip_data_file_movie, 'r') as zip_ref:
        zip_ref.extractall(resources_dir)

if not exists(database_path_pitchfork):
    with ZipFile(zip_data_file_pitchfork, 'r') as zip_ref:
        zip_ref.extractall(resources_dir)

Funkcje pomocnicze do obsługi bazy danych SQLite3

In [5]:
from typing import List, Tuple
import sqlite3

def get_table_columns_types(db_cursor: sqlite3.Cursor, table_name: str) -> List[Tuple[str, str]]:
    """Returns a list of tuples containing the column name and data type of given table in a database."""

    info = db_cursor.execute(f"""
        PRAGMA
            table_info({table_name});
    """).fetchall()

    return list((table_info[1], table_info[2]) for table_info in info)

def get_table_names(db_cursor: sqlite3.Cursor) -> List[str]:
    """"""
    db_cursor.execute("""
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type='table'
    """)

    return list(table[0] for table in db_cursor.fetchall())

def create_table_with_primary_key(db_cursor: sqlite3.Cursor, table_name: str, temporary_table_name: str, column_name: str) -> None:
    """Creates a table with a primary key in a database."""

    table_columns = get_table_columns_types(db_cursor, table_name)

    db_cursor.execute(f"""
        CREATE TABLE {temporary_table_name} (
            {
                ', '.join(
                    f'{column} {data_type}'
                    if column != column_name
                    else f'{column} {data_type} PRIMARY KEY'
                    for column, data_type in table_columns
                )
            }
        );
    """)

def move_data_and_switch_tables(db_cursor: sqlite3.Cursor, table_name: str, temporary_table_name: str) -> None:
    """Moves data from a temporary table to the original table and switches the tables in a database."""

    db_cursor.executescript(f"""
        -- Move the data
        INSERT INTO
            {temporary_table_name}
        SELECT
            *
        FROM
            {table_name};
        
        -- Drop the old table
        DROP TABLE
            {table_name};
        
        -- Rename the new table
        ALTER TABLE
            {temporary_table_name}
        RENAME TO
            {table_name};
    """)

def create_table_with_foreign_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str, foreign_table_name: str, foreign_column_name: str, temporary_foreign_table_name: str) -> None:
    """Creates a table with a foreign key in a database."""

    foreign_table_columns = get_table_columns_types(db_cursor, foreign_table_name)

    db_cursor.execute(f"""
        CREATE TABLE {temporary_foreign_table_name} (
            {
                ', '.join(
                    f'{column} {data_type}'
                    for column, data_type in foreign_table_columns
                )
            },
            CONSTRAINT fk_{foreign_table_name}_{foreign_column_name}
                FOREIGN KEY
                    ({foreign_column_name})
                REFERENCES
                    {table_name} ({column_name})
        );
    """)

Limitowanie danych w tabelach do 1000 rekordów

In [6]:
def delete_duplicates(db_cursor: sqlite3.Cursor, table_name: str, column_name: str) -> None:
    """Deletes duplicates from a given table column in a database."""


    for given_table_name in get_table_names(db_cursor):
        for given_column_name, _ in get_table_columns_types(db_cursor, given_table_name):
            if column_name == given_column_name:
                db_cursor.execute(f"""
                    DELETE FROM
                        {given_table_name}
                    WHERE
                        {given_column_name} IN (
                            SELECT
                                {column_name}
                            FROM
                                {table_name}
                            GROUP BY
                                {column_name}
                            HAVING
                                COUNT(*) > 1
                        );
                """)
            else:
                db_cursor.execute(f"""
                    DELETE FROM
                        {given_table_name}
                    WHERE
                        {given_column_name} IS NULL;
                """)

    db_cursor.execute(f"""
        DELETE FROM
            {table_name}
        WHERE
            {column_name} NOT IN (
                {
    'INTERSECT'.join(f"""
                            SELECT
                                {column_name}
                            FROM
                                {given_table_name}
                        """
                     for given_table_name in get_table_names(db_cursor)
                     )
    }
            );
    """)

    for given_table_name in get_table_names(db_cursor):
        db_cursor.execute(f"""
            DELETE FROM
                {given_table_name}
            WHERE
                {column_name} NOT IN (
                    SELECT
                        {column_name}
                    FROM
                        {table_name}
                );
        """)

In [7]:
def limit_rows(db_cursor: sqlite3.Cursor, column_name: str, max_rows: int) -> None:
    """Limits the number of rows in a table in a database."""
    
    all_tables = get_table_names(db_cursor)
    
    # From every table and every column deletes rows that are NULL
    for table_name in all_tables:
        for given_column_name, _ in get_table_columns_types(db_cursor, table_name):
            db_cursor.execute(f"""
                DELETE FROM
                    {table_name}
                WHERE
                    {given_column_name} IS NULL;
            """)
    
    # From every table delete rows that are a duplicate of the column_name
    for table_name in all_tables:
        db_cursor.execute(f"""
            DELETE FROM
                {table_name}
            WHERE
                {column_name} IN (
                    SELECT
                        {column_name}
                    FROM
                        {table_name}
                    GROUP BY
                        {column_name}
                    HAVING
                        COUNT(*) > 1
                );
        """)
    
    # Find the intersection of a column_name in every table
    # Delete rows that are not in the intersection
    intersection: str = 'INTERSECT'.join(f"""
        SELECT
            {column_name}
        FROM
            {intersection_table_name}
    """ for intersection_table_name in all_tables)
    
    for table_name in all_tables:
        db_cursor.execute(f"""
            DELETE FROM
                {table_name}
            WHERE
                {column_name} NOT IN (
                    {intersection}
                );
        """)
    
    # Limit the number of rows in a table to max_rows
    for table_name in all_tables:
        db_cursor.execute(f"""
            DELETE FROM
                {table_name}
            WHERE
                {column_name} NOT IN (
                    SELECT
                        {column_name}
                    FROM
                        {table_name}
                    ORDER BY
                        {column_name}
                    LIMIT {max_rows}
                );
        """)

In [8]:
from typing import Final

MAX_ROWS: Final[int] = 1000

with sqlite3.connect(database_path_movie) as conn:
    limit_rows(conn.cursor(), 'Movie_id', MAX_ROWS)

In [9]:
with sqlite3.connect(database_path_pitchfork) as conn:
    limit_rows(conn.cursor(), 'reviewid', MAX_ROWS)

Dodanie kluczy głównych i obcych do bazy danych, ponieważ nie zostały one dodane podczas tworzenia na platformie Kaggle, a jest to ważne podczas tworzenia MAGN.

In [10]:
def add_foreign_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str, foreign_table_name: str, foreign_column_name: str) -> None:
    """Adds a foreign key to a given table column in a database."""

    temporary_foreign_table_name = f'{foreign_table_name}_'

    db_cursor.execute(f"""
        PRAGMA
            foreign_keys = OFF;
    """)

    create_table_with_foreign_key(db_cursor, table_name, column_name, foreign_table_name, foreign_column_name, temporary_foreign_table_name)
    move_data_and_switch_tables(db_cursor, foreign_table_name, temporary_foreign_table_name)

    db_cursor.execute(f"""
        PRAGMA
            foreign_keys = ON;
    """)

def add_primary_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str) -> None:
    """Adds a primary key to a given table column in a database."""

    temporary_table_name = f'{table_name}_'

    create_table_with_primary_key(db_cursor, table_name, temporary_table_name, column_name)
    move_data_and_switch_tables(db_cursor, table_name, temporary_table_name)

In [11]:
with sqlite3.connect(database_path_movie) as conn:
    cursor = conn.cursor()

    cursor.execute(f"""BEGIN;""")
    try:
        add_primary_key(cursor, 'IMDB', 'Movie_id')

        add_foreign_key(cursor, 'IMDB', 'Movie_id', 'earning', 'Movie_id')
        add_foreign_key(cursor, 'IMDB', 'Movie_id', 'genre', 'Movie_id')
    except Exception as error:
        conn.rollback()
        raise error
    else:
        conn.commit()

![ERD Diagram for the database schema](docs/images/database_erd_movie.png_)

In [12]:
with sqlite3.connect(database_path_pitchfork) as conn:
    cursor = conn.cursor()

    cursor.execute(f"""BEGIN;""")
    try:
        add_primary_key(cursor, 'reviews', 'reviewid')

        add_foreign_key(cursor, 'reviews', 'reviewid', 'artists', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'content', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'genres', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'labels', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'years', 'reviewid')
    except Exception as error:
        conn.rollback()
        raise error
    else:
        conn.commit()

![ERD Diagram for the database schema](docs/images/database_erd_pitchfork.png_)

In [13]:
from magn.database.sqlite3 import get_table_names

all_tables_movie = get_table_names(database_path_movie)

all_tables_movie

['IMDB', 'earning', 'genre']

In [14]:
all_tables_pitchfork = get_table_names(database_path_pitchfork)

all_tables_pitchfork

['reviews', 'artists', 'content', 'genres', 'labels', 'years']

In [15]:
from magn.database.sqlite3 import SQLite3KeysReader

keys_reader = SQLite3KeysReader(database_path_movie, all_tables_movie)
keys_movie = keys_reader.read()

keys_movie

{'IMDB': Keys(primary_keys=['Movie_id'], foreign_keys={}),
 'earning': Keys(primary_keys=[], foreign_keys={'IMDB': ('Movie_id', 'Movie_id')}),
 'genre': Keys(primary_keys=[], foreign_keys={'IMDB': ('Movie_id', 'Movie_id')})}

In [16]:
keys_reader = SQLite3KeysReader(database_path_pitchfork, all_tables_pitchfork)
keys_pitchfork = keys_reader.read()

keys_pitchfork

{'reviews': Keys(primary_keys=['reviewid'], foreign_keys={}),
 'artists': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'content': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'genres': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'labels': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'years': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')})}

In [17]:
from magn.database.sqlite3 import SQLite3DataReader

data_reader = SQLite3DataReader(database_path_movie, all_tables_movie, keys_movie)
data_reader.read()

{'IMDB': Empty DataFrame
 Columns: [Title, Rating, TotalVotes, MetaCritic, Budget, Runtime, CVotes10, CVotes09, CVotes08, CVotes07, CVotes06, CVotes05, CVotes04, CVotes03, CVotes02, CVotes01, CVotesMale, CVotesFemale, CVotesU18, CVotesU18M, CVotesU18F, CVotes1829, CVotes1829M, CVotes1829F, CVotes3044, CVotes3044M, CVotes3044F, CVotes45A, CVotes45AM, CVotes45AF, CVotes1000, CVotesUS, CVotesnUS, VotesM, VotesF, VotesU18, VotesU18M, VotesU18F, Votes1829, Votes1829M, Votes1829F, Votes3044, Votes3044M, Votes3044F, Votes45A, Votes45AM, Votes45AF, VotesIMDB, Votes1000, VotesUS, VotesnUS]
 Index: []
 
 [0 rows x 51 columns],
 'earning': Empty DataFrame
 Columns: [Domestic, Worldwide]
 Index: [],
 'genre': Empty DataFrame
 Columns: [genre]
 Index: []}

In [18]:
data_reader = SQLite3DataReader(database_path_pitchfork, all_tables_pitchfork, keys_pitchfork)
data_reader.read()

{'reviews':                                   title           artist  \
 reviewid                                                   
 1                         young forever        aberfeldy   
 6                  pure tone audiometry         aarktica   
 11        homesick and happy to be here         aberdeen   
 31                          rolled gold           action   
 33             the scene's out of sight     actionslacks   
 ...                                 ...              ...   
 5329                 advisory committee            mirah   
 5334                radio-free brooklyn       pete miser   
 5339                          bangzilla  mix master mike   
 5341                           three ep      mobius band   
 5342                 city vs country ep      mobius band   
 
                                                         url  score  \
 reviewid                                                             
 1         http://pitchfork.com/reviews/albums/1-you

In [19]:
from magn.database.database import Database

database_movie = Database.from_sqlite3(database_path_movie)

database_movie

Database(all_data={'IMDB': Table(data=Empty DataFrame
Columns: [Title, Rating, TotalVotes, MetaCritic, Budget, Runtime, CVotes10, CVotes09, CVotes08, CVotes07, CVotes06, CVotes05, CVotes04, CVotes03, CVotes02, CVotes01, CVotesMale, CVotesFemale, CVotesU18, CVotesU18M, CVotesU18F, CVotes1829, CVotes1829M, CVotes1829F, CVotes3044, CVotes3044M, CVotes3044F, CVotes45A, CVotes45AM, CVotes45AF, CVotes1000, CVotesUS, CVotesnUS, VotesM, VotesF, VotesU18, VotesU18M, VotesU18F, Votes1829, Votes1829M, Votes1829F, Votes3044, Votes3044M, Votes3044F, Votes45A, Votes45AM, Votes45AF, VotesIMDB, Votes1000, VotesUS, VotesnUS]
Index: []

[0 rows x 51 columns], keys=Keys(primary_keys=['Movie_id'], foreign_keys={})), 'earning': Table(data=Empty DataFrame
Columns: [Domestic, Worldwide]
Index: [], keys=Keys(primary_keys=[], foreign_keys={'IMDB': ('Movie_id', 'Movie_id')})), 'genre': Table(data=Empty DataFrame
Columns: [genre]
Index: [], keys=Keys(primary_keys=[], foreign_keys={'IMDB': ('Movie_id', 'Movie_id'

In [20]:
database_pitchfork = Database.from_sqlite3(database_path_pitchfork)

database_pitchfork

Database(all_data={'reviews': Table(data=                                  title           artist  \
reviewid                                                   
1                         young forever        aberfeldy   
6                  pure tone audiometry         aarktica   
11        homesick and happy to be here         aberdeen   
31                          rolled gold           action   
33             the scene's out of sight     actionslacks   
...                                 ...              ...   
5329                 advisory committee            mirah   
5334                radio-free brooklyn       pete miser   
5339                          bangzilla  mix master mike   
5341                           three ep      mobius band   
5342                 city vs country ep      mobius band   

                                                        url  score  \
reviewid                                                             
1         http://pitchfork.com/reviews

In [21]:
database_pitchfork.create_mock_target("reviews")

,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,target
reviewid,,,,,,,,,,,,,
1,young forever,aberfeldy,http://pitchfork.com/reviews/albums/1-young-fo...,3.4,0,marc hogan,senior staff writer,2004-08-30,0,30,8,2004,author_type
6,pure tone audiometry,aarktica,http://pitchfork.com/reviews/albums/6-pure-ton...,7.4,0,eric carr,contributor,2003-04-08,1,8,4,2003,author_type
11,homesick and happy to be here,aberdeen,http://pitchfork.com/reviews/albums/11-homesic...,7.0,0,joe tangari,contributor,2002-11-04,0,4,11,2002,title
31,rolled gold,action,http://pitchfork.com/reviews/albums/31-rolled-...,8.0,0,dominique leone,contributor,2002-11-20,2,20,11,2002,best_new_music
33,the scene's out of sight,actionslacks,http://pitchfork.com/reviews/albums/33-the-sce...,7.6,0,ryan schreiber,editor-in-chief,2000-12-31,6,31,12,2000,pub_weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5329,advisory committee,mirah,http://pitchfork.com/reviews/albums/5329-advis...,8.3,0,matt lemay,contributor,2002-04-08,0,8,4,2002,title
5334,radio-free brooklyn,pete miser,http://pitchfork.com/reviews/albums/5334-radio...,7.2,0,julianne escobedo shepherd,contributor,2003-04-07,0,7,4,2003,best_new_music
5339,bangzilla,mix master mike,http://pitchfork.com/reviews/albums/5339-bangz...,5.0,0,peter macia,contributor,2004-12-07,1,7,12,2004,pub_month
